In [1]:
# %pip install torch
%pip install torchtext
# import torch
# from torch import nn
# from torchtext.data.utils import get_tokenizer
# from torchtext.vocab import build_vocab_from_iterator
# from torchtext.legacy.data import Field, TabularDataset, BucketIterator

import os
import re
import shutil
import string
import nltk

from collections import Counter


import pandas as pd
import numpy as np

import sklearn


from sklearn.model_selection import train_test_split


Note: you may need to restart the kernel to use updated packages.


In [2]:
import nltk
import spacy
import numpy as np

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


In [21]:
# Basic text cleanup
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

def remove_url(text): 
    url_pattern  = re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    return url_pattern.sub(r'', text)
 # converting return value from list to string

def clean_text(text): 
    delete_dict = {sp_character: '' for sp_character in string.punctuation} 
    delete_dict[' '] = ' ' 
    table = str.maketrans(delete_dict)
    text1 = text.translate(table)
    #print('cleaned:'+text1)
    textArr= text1.split()
    text2 = ' '.join([w for w in textArr if ( not w.isdigit() and  ( not w.isdigit() and len(w)>2))]) 
    return text2.lower()


def remove_stopwords(message_text):
    """Remove any stopwords that bear no meaning"""
    stop_words = set(stopwords.words('english'))
    try:
        word_tokens = word_tokenize(message_text)
        filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
        filtered_sentence = " ".join(filtered_sentence)
        return filtered_sentence
    except IndexError:
        return message_text
        


def remove_reply_lines(message_text):
    """Try and remove as many reply lines as we can"""
    message_text = "".join(line for line in message_text if not line.startswith('>'))
    return message_text


def remove_non_english(message_text):
    """Remove anything remotely not English. If it is not in English, it should not be processed in ANY form"""
    words = set(nltk.corpus.words.words())
    filtered_sentence = " ".join(w for w in nltk.wordpunct_tokenize(message_text) \
                                 if w.lower() in words or not w.isalpha())
    return filtered_sentence


def remove_emails(message_text):
    """Finally, remove any links, emails and other non-essential trash from the text"""
    text = re.sub(r"\S*@\S*\s?", '', message_text, flags=re.MULTILINE)
    text = re.sub(r"<.*?>", '', text, flags=re.MULTILINE)
    return text


In [22]:
train_data= pd.read_csv("/Users/yaroslavkizyma/Desktop/json_data_exports_alternative/encoded_data_alternative_train.csv")
train_data.dropna(axis = 0, how ='any', inplace=True) 
train_data['Num_words_text'] = train_data['message'].apply(lambda x:len(str(x).split())) 
mask = train_data['Num_words_text'] > 0
train_data = train_data[mask]
print('-------Train data--------')
print(train_data['tag'].value_counts())
print(len(train_data))
print('-------------------------')
max_train_sentence_length  = train_data['Num_words_text'].max()


train_data['message'] = train_data['message'].apply(remove_emoji)
train_data['message'] = train_data['message'].apply(remove_stopwords)
# train_data['message'] = train_data['message'].apply(remove_reply_lines)
train_data['message'] = train_data['message'].apply(remove_non_english)
train_data['message'] = train_data['message'].apply(remove_emails)
train_data['message'] = train_data['message'].apply(remove_url)
train_data['message'] = train_data['message'].apply(clean_text)


test_data= pd.read_csv("/Users/yaroslavkizyma/Desktop/json_data_exports_alternative/encoded_data_alternative_test.csv")
test_data.dropna(axis = 0, how ='any', inplace=True) 
test_data['Num_words_text'] = test_data['message'].apply(lambda x:len(str(x).split())) 

max_test_sentence_length  = test_data['Num_words_text'].max()

mask = test_data['Num_words_text'] > 0
test_data = test_data[mask]

print('-------Test data--------')
print(test_data['tag'].value_counts())
print(len(test_data))
print('-------------------------')

test_data['message'] = test_data['message'].apply(remove_emoji)
test_data['message'] = test_data['message'].apply(remove_stopwords)
# test_data['message'] = test_data['message'].apply(remove_reply_lines)
test_data['message'] = test_data['message'].apply(remove_non_english)
test_data['message'] = test_data['message'].apply(remove_emails)
test_data['message'] = test_data['message'].apply(remove_url)
test_data['message'] = test_data['message'].apply(clean_text)

print('Train Max Message Length :'+str(max_train_sentence_length))
print('Test Max Message Length :'+str(max_test_sentence_length))

-------Train data--------
3     800
7     800
1     799
5     799
6     719
8     582
11    482
2     362
9     350
4     278
10    268
Name: tag, dtype: int64
6239
-------------------------
-------Test data--------
3     200
5     200
7     200
1     199
6     180
8     146
11    120
2      91
9      88
4      69
10     68
Name: tag, dtype: int64
1561
-------------------------
Train Max Message Length :33468
Test Max Message Length :10910


In [23]:
train_data.head(10)

,message,tag,Num_words_text
0,unfortunately link link kind file program reca...,8,116
1,neo order price need conformation address link...,8,45
2,dear sir hit export button nothing seen progre...,11,148
3,dear finally save attached let know need anyth...,5,438
4,name description hello expiration date members...,4,38
5,said partial refund would like full refund ple...,7,97
6,hello hope well thank know issue ending please...,9,81
7,hello thank like refund please support thanks ...,7,176
8,message help sure mean working recently tried ...,5,437
9,good morning understand entirely without quest...,5,313


In [24]:
X_train, X_valid, Y_train, Y_valid = train_test_split(train_data['message'].tolist(),\
                                                      train_data['tag'].tolist(),\
                                                      test_size=0.2,\
                                                      stratify = train_data['tag'].tolist(),\
                                                      random_state=0)


print('Train data len:'+str(len(X_train)))
print('Class distribution '+str(Counter(Y_train)))


print('Valid data len:'+str(len(X_valid)))
print('Class distribution '+ str(Counter(Y_valid)))

print('Test data len:'+str(len(test_data['message'].tolist())))
print('Class distribution '+ str(Counter(test_data['tag'].tolist())))


train_dat =list(zip(Y_train,X_train))
valid_dat =list(zip(Y_valid,X_valid))
test_dat=list(zip(test_data['tag'].tolist(),test_data['message'].tolist()))

Train data len:4991
Class distribution Counter({7: 640, 3: 640, 1: 639, 5: 639, 6: 575, 8: 466, 11: 386, 2: 290, 9: 280, 4: 222, 10: 214})
Valid data len:1248
Class distribution Counter({3: 160, 5: 160, 7: 160, 1: 160, 6: 144, 8: 116, 11: 96, 2: 72, 9: 70, 4: 56, 10: 54})
Test data len:1561
Class distribution Counter({3: 200, 5: 200, 7: 200, 1: 199, 6: 180, 8: 146, 11: 120, 2: 91, 9: 88, 4: 69, 10: 68})


In [25]:
# redo into a function proper

for tuple_elem in train_dat:
    class_tag, text = tuple_elem
    if len(text) == 0:
        train_dat.remove(tuple_elem)
        print(f"Empty elem removed")

for tuple_elem in test_dat:
    class_tag, text = tuple_elem
    if len(text) == 0:
        test_dat.remove(tuple_elem)
        print(f"Empty elem removed")


for tuple_elem in valid_dat:
    class_tag, text = tuple_elem
    if len(text) == 0:
        valid_dat.remove(tuple_elem)
        print(f"Empty elem removed")

Empty elem removed
Empty elem removed
Empty elem removed
Empty elem removed
Empty elem removed
Empty elem removed
Empty elem removed
Empty elem removed
Empty elem removed
Empty elem removed
Empty elem removed
Empty elem removed
Empty elem removed


In [26]:
import torch
from torch.utils.data import DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [27]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

tokenizer = get_tokenizer('basic_english')
train_iter = train_dat
def yield_tokens(data_iter):
    for _, text in data_iter:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

In [28]:
text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: int(x) - 1


In [29]:
text_pipeline('here is the an example')


[309, 0, 1084, 0, 425]

In [30]:
label_pipeline('1')


0

In [31]:
def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
         label_list.append(label_pipeline(_label))
         processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
         text_list.append(processed_text)
         offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)

#train_iter = train_dat
#dataloader = DataLoader(train_iter, batch_size=8, shuffle=False, collate_fn=collate_batch)

In [32]:
# class LSTM_variable_input(torch.nn.Module) :
#     def __init__(self, vocab_size, embedding_dim, hidden_dim) :
#         super().__init__()
#         self.hidden_dim = hidden_dim
#         self.dropout = nn.Dropout(0.3)
#         self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
#         self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
#         self.linear = nn.Linear(hidden_dim, 5)
        
#     def forward(self, x, s):
#         x = self.embeddings(x)
#         x = self.dropout(x)
#         x_pack = pack_padded_sequence(x, s, batch_first=True, enforce_sorted=False)
#         out_pack, (ht, ct) = self.lstm(x_pack)
#         out = self.linear(ht[-1])
#         return out

In [37]:
class LSTM_fixed_len(torch.nn.Module) :
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_class) :
        super().__init__()
        self.embedding_dim = embedding_dim
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=3)
        self.linear = nn.Linear(hidden_dim, num_class)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x, l):
        len_ = len(x)
        if len_ < 1:
            print(len_)
        x = self.embeddings(x)
        x = self.dropout(x)
        lstm_out, (ht, ct) = self.lstm(x.view(len_, 1, -1))
        
        return self.linear(lstm_out[-1])

In [38]:
import torch.nn as nn

train_iter1 = train_dat
num_class = len(set([label for (label, text) in train_iter1]))
print(num_class)
vocab_size = len(vocab)
print(vocab_size)
# embedding_size
emsize = 16
# hidden_dim
h_dim = 16
# model = LSTM_variable_input(vocab_size, emsize, num_class)
model =  LSTM_fixed_len(vocab_size, emsize, h_dim, num_class)

# model = TextClassificationModel(vocab_size, emsize, num_class).to(device)


11
7941


In [39]:
import time

def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predited_label = model(text, offsets)
        loss = criterion(predited_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predited_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predited_label = model(text, offsets)
            loss = criterion(predited_label, label)
            total_acc += (predited_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

In [40]:
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset
# Hyperparameters
EPOCHS = 25 # epoch
LR = 0.01 # learning rate
# BATCH_SIZE = 16 # batch size for training
BATCH_SIZE = 1 # batch size for training


criterion = torch.nn.CrossEntropyLoss()
# optimizer = torch.optim.SGD(model.parameters(), lr=LR)
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None

train_iter2 = train_dat
test_iter2 = test_dat 
valid_iter2= valid_dat




train_dataloader = DataLoader(train_iter2, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(valid_iter2, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_iter2, batch_size=BATCH_SIZE,
                             shuffle=True, collate_fn=collate_batch)


# for index, data in enumerate(train_dataloader):
#     if len(data[1]) == 0:
#         print('Alarm')
#         print(index)

for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
      scheduler.step()
    else:
       total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))
    print('-' * 59)

| epoch   1 |   500/ 4982 batches | accuracy    0.140
| epoch   1 |  1000/ 4982 batches | accuracy    0.152
| epoch   1 |  1500/ 4982 batches | accuracy    0.160
| epoch   1 |  2000/ 4982 batches | accuracy    0.156
| epoch   1 |  2500/ 4982 batches | accuracy    0.154
| epoch   1 |  3000/ 4982 batches | accuracy    0.174
| epoch   1 |  3500/ 4982 batches | accuracy    0.186
| epoch   1 |  4000/ 4982 batches | accuracy    0.188
| epoch   1 |  4500/ 4982 batches | accuracy    0.192
-----------------------------------------------------------
| end of epoch   1 | time: 98.34s | valid accuracy    0.219 
-----------------------------------------------------------
| epoch   2 |   500/ 4982 batches | accuracy    0.204
| epoch   2 |  1000/ 4982 batches | accuracy    0.220
| epoch   2 |  1500/ 4982 batches | accuracy    0.248
| epoch   2 |  2000/ 4982 batches | accuracy    0.230
| epoch   2 |  2500/ 4982 batches | accuracy    0.234
| epoch   2 |  3000/ 4982 batches | accuracy    0.226
| epoch  

In [ ]:
type(train_dataloader)